Импорт необходимых библиотек (pandas для обработки данных и scikit-learn для моделирования).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Загрузка и подготовка данных (их разделение на контрольную и экспериментальную группы).

In [ ]:
data = pd.read_csv('data.csv')

# значением 0 отмечаем тех пользователей, которые не получили воздействие, 
# и значением 1 - остальных.
data['treatment'] = (data["group"] != "control").astype("int")

Создание признаков и целевой переменной.

In [ ]:
X = data[['age', 'gender', 'location', 'previous_purchases', 'treatment']]
y = data['response']

Разделение данных на обучающую и тестовую выборки.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Обучение модели на обучающей выборке.

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

Оценка модели на тестовой выборке с помощью соответствующих метрик

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

************************************************************************************************

In [3]:
# загружаем необходимые библиотеки
import warnings

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# загружаем датафрейм
df = pd.read_csv("discountuplift.csv", sep="\t")

df["old_target"] = (df["target_class"] % 2).apply(int)

feature_cols = [
    "recency",
    "history",
    "used_discount",
    "used_bogo",
    "is_referral",
    "zip_code_Rural",
    "zip_code_Surburban",
    "zip_code_Urban",
    "channel_Multichannel",
    "channel_Phone",
    "channel_Web",
]
target_col = "old_target"
treatment_col = "treatment"

# разобъём нашу выборку на тестовую и валидационную
df_train, df_test = train_test_split(
    df, stratify=df[[treatment_col, target_col]], random_state=1, test_size=0.25
)


In [4]:
from causalml.inference.tree import UpliftTreeClassifier

# создадим uplift-дерево
uplift_model = UpliftTreeClassifier(
    max_depth=5,
    min_samples_leaf=200,
    min_samples_treatment=50,
    n_reg=100,
    evaluationFunction="ED",
    control_name="0",
)

uplift_model.fit(
    df_train[feature_cols].values,
    treatment=df_train[treatment_col].apply(str).values,
    y=df_train[target_col].values,
)

Failed to import duecredit due to No module named 'duecredit'


In [5]:
# загружаем необходимые библиотеки
import pickle

with open("uplift_model.pkl", "wb") as f:
    pickle.dump(uplift_model, f)

далее делаю микросервис uplift_main.py

в терминале

 - cd ~/mle_projects/mle-recsys-start/
 - source .venv_sprint4/bin/activate

 - cd ~/mle_projects/mle-sprint-5_example/
 - uvicorn uplift_main:app --host 0.0.0.0 --port 5000 


In [8]:
import requests

feature_cols = [
    "recency",
    "history",
    "used_discount",
    "used_bogo",
    "is_referral",
    "zip_code_Rural",
    "zip_code_Surburban",
    "zip_code_Urban",
    "channel_Multichannel",
    "channel_Phone",
    "channel_Web",
]

# пример данных для запроса
data = {
    "features": df[feature_cols].sample(1).values.tolist()
}

# URL сервиса
url = "http://localhost:5000/predict"

# выполнение POST-запроса
response = requests.post(url, json=data)

# проверка ответа
if response.status_code == 200:
    print("Ответ сервера:", response.json())
else:
    print("Ошибка:", response.status_code, response.text)

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9f2ab4fdc0>: Failed to establish a new connection: [Errno 111] Connection refused'))

создать файл requirements.txt

Uplift_DockerFile

In [ ]:
#cd ~/mle_projects/mle-sprint-5_example/

# Соберём его:
# docker build -t uplift-model . 

# Запустим:
# docker run -p 5000:5000 uplift-model 

# Итак, сервис поднялся. Снова можно протестировать его запросом. Для разнообразия сделаем это с помощью curl:
# curl -X POST -H "Content-Type: application/json" -d '{"features": [10, 95.49,  0,  1,  1,  1,  0,  0,  0, 0,  1]}' http://localhost:5000/predict 






# Запуск Graphite из готового образа graphiteapp/graphite-statsd. Пояснения портов:
#     80: административный веб-интерфейс,
#     2003/8125: порты для сбора метрик по разным протоколам (newline delimited, pickle),
#     7002: порт для получения метрик.
# docker run -d \
#   --name graphite \
#   -p 80:80 \
#   -p 2003:2003 \
#   -p 8125:8125/udp \
#   -p 8126:8126 \
#   graphiteapp/graphite-statsd
# После запуска Graphite вы можете открыть его веб-интерфейс по адресу http://<ip address вашей ВМ>:80/.

# Запуск нашего сервиса.
# docker build -t uplift-model .
# docker run -p 5000:5000 --link graphite1:graphite uplift-model 

# Вы можете протестировать сервис, отправив запрос через curl или python:
# curl -X POST -H "Content-Type: application/json" \
# -d '{"features": [10, 95.49,  0,  1,  1,  1,  0,  0,  0, 0,  1]}' \
# http://localhost:5000/predict 